# 基于医学规则的排牙与分步

该Notebook将演示：

1. 如何调用朝厚云接口生成自动加工单并根据自动加工单生成符合要求的排牙
2. 如何根据医生手动修改或编写的加工单自动生成符合加工单要求的排牙
3. 如何根据排牙生成分步方案

使用本Notebook需要准备一个病例的：
1. 侧位片
2. 微笑照
3. 上下颌口扫模型

In [ ]:
# 导入必要的包以及定义函数
import os
import time
import requests
import json
import trimesh
import urllib
import numpy as np

# 定义调用规则

请根据您从我方获取的信息修改以下代码块

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 朝厚文件服务地址，随api文档发送
file_server_url = "<服务文件服务器地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

user_group = "APIClient" # 用户组，一般为 APIClient

# 贵司user_id, 随api文档发送
user_id = "<贵司user_id>"

# 如果您收到了creds.json, 下面将直接读取
if os.path.exists('creds.json'):
    creds = json.load(open('creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
# 定义调用JSON的基础部分 - 调用的服务名称以及您的用户名信息
json_call = {
  "spec_group": "mesh-processing", # 调用的工作流组， 随API文档发送
  "spec_name": "oral-arrangement-medical", # 调用的工作流名称， 随API文档发送, 
  "spec_version": "1.0-snapshot", # 调用的工作流版本，随API文档发送
  "user_group": user_group,
  "user_id": user_id
}

### 定义回调地址

回调会在工作流运行结束后向指定地址发送一个POST请求，请求内容为一个JSON，内含4个字段

1. workflow_id (str): 对应的workflow_id
2. metadata (dict): 您启动工作流时传入的metadata
3. success (bool): 工作流是否成功（true为成功）
4. reason (str or null): 如果success为true, 该项为null. 否则为失败原因的字符串表示

如果需要回调，请uncomment以下代码块

In [ ]:
# json_call['metadata'] = { #（可选）这里可以加入metadata, 回调时会附带该信息，字典每项限制128个字符
#     "case_id": "CH-123",
#     "case_name": "ABCDE"
# }
# json_call['notification'] =[ #（可选）这里可以加入回调地址
#     {"url": "https://www.baidu.com"} # 这里可以加入多个回调地址，每个都是{"url": "xxx"}格式
# ]

## 定义输入/输出块

输入块由JSON文件中的`input_data`定义。对于三维网格，我们支持以下**输入**文件格式

```
"obj"
"stl"
"off"
"ply"
"glb"
"zip": zip内必须有且仅有一个mesh文件，且其后缀名必须为 obj, stl, off, ply 或 glb
"tar.gz": tar 或者 tar.gz内必须有且仅有一个mesh文件，且其后缀名必须为 obj, stl, off, ply 或 glb
"drc"
```

和以下**输出**文件格式
```
"obj"
"stl"
"off"
"ply"
"glb"
"drc"
```


我们**强烈推荐**在输入/输出中使用`drc`文件格式以减少网络传输时间，生成和读取drc格式文件可以使用Google的Draco库: https://github.com/google/draco

下表显示了同一个三维网格在不同文件格式下的文件大小 (点精度： 0.001mm）

| 文件格式 | 文件大小 |
| --- | --- |
| stl | 16.7M |
| obj | 13.5M |
| off | 14.2M |
| ply | 7.0M |
| glb | 6.0M |
| drc | 517.3K |


对于三维网格，输入可以采用两种形式传入：

1. 先将文件传入我们的文件服务系统，再将文件指针写入JSON调用串并调用API
2. 直接在API的JSON调用串中传入base64编码后的二进制数据

对于三维网格，输出可以指定两种形式传出：

1. 在API数据接口中返回文件指针，用户去文件系统下载具体二进制文件
2. 直接在API的数据接口中将二进制数据以base64编码形式返回


我们**强烈推荐**使用第一种方式，理由如下：

1. base64编码将增大数据流大小，提高网络延迟
2. 为保障接口性能，朝厚将拒绝过大的API请求。因此对于大文件，第二种方式会调用失败（HTTP CODE 413)

### 使用文件方式上传

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open(file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # 必须指定 postfix, 即文件后缀名
                        headers={"X-ZH-TOKEN": zh_token}) # 获取带签名的上传地址
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # 返回为一个单字符串JSON "string", 这里也可以用json.loads(resp.text)

    resp = requests.put(upload_url, data) # 上传至云储存服务不需要带鉴权头

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

json_call["input_data"] = { # 输入数据，此处依照不同的任务字典内容会有不同，具体参照API文档，该处以分牙+牙轴举例
    "upper_mesh": {"type":"ply", "data": upload_file('u.ply')},
    "lower_mesh": {"type":"ply", "data": upload_file('l.ply')},
    "ceph": upload_file("ceph.jpg"),
    "smile_photo": upload_file("smile.jpg")
}

# 定义输出配置（输出与输入不一定需要一样，比如可以输入文件stl, 输出base64的drc
json_call['output_config'] = {
    "arranged_comp": {"type": "ply"},
    "teeth_comp": {"type": "ply"}
}

# 提交请求

使用 `/run` POST方法提交请求

In [ ]:
headers = {
  "Content-Type": "application/json",
  "X-ZH-TOKEN": zh_token
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

# 打印任务号
print(run_id)

# 等待请求完成

使用轮询API等待请求完成，API地址为 `/run/{run_id}`

**强烈推荐使用回调方法**, 参考[定义回调地址](#定义回调地址)，回调方法无论工作流成功与否，在结束时均会向给定地址发送回调信息，您无需轮询结果

In [ ]:
# 第二步： 轮询状态
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 600: # 最多等10分钟
    time.sleep(3) # 轮询间隔
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break
    
if not result['completed']:
    if result['failed']:
        raise ValueError("API运行错误，原因： " + str(result['reason_public']))
    raise TimeoutError("API超时")

print("API运行时间： {}s".format(time.time()-start_time))

# 获取请求结果

使用 `/data/{run_id}` 获得该工作流所有输出数据，使用 `/data/{run_id}/{key}` 获取某一项数据

In [ ]:
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

In [ ]:
def retrieve_mesh(mesh_file_json):
    resp = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": mesh_file_json['data']}),
                        headers={"X-ZH-TOKEN": zh_token})
    return trimesh.load(trimesh.util.wrap_as_stream(resp.content), file_type=mesh_file_json['type'])

arranged_comp = {fdi: retrieve_mesh(x) for fdi, x in result['arranged_comp'].items()}
original_comp = {fdi: retrieve_mesh(x) for fdi, x in result['teeth_comp'].items()}

# 样例API输出结果展示

以下输出为自动生成的工单

In [ ]:
print(json.loads(result['form']))

将工单输出为自然语言

In [ ]:
def form2str(form):
    def collision2str(collisions):
        collision_name_dict = {
            'left_back_teeth_IPR': '左侧后牙IPR',
            'front_teeth_IPR': '前牙IPR',
            'right_back_teeth_IPR': '右侧后牙IPR',
            'move_left_molar': '远移左侧磨牙',
            'move_right_molar': '远移右侧磨牙',
            'move_both_molar_forward': '磨牙近移'
        }

        return '，'.join(
            [collision_name_dict[
                 collision[0] if isinstance(collision, tuple) or isinstance(collision, list) else collision] for
             collision in
             collisions])

    cusp_name_dict = {
        'near_cusp': '近中切角',
        'mid_cusp': '切端',
        'far_cusp': '远中切角'
    }
    direction_name_dict = {
        'backward': '内收',
        'forward': '唇倾',
        'down': '压低',
        'up': '伸长',
        'left': '左移',
        'right': '右移'
    }

    form_str = f"""
    下颌目标位
    1. 前牙矢状向目标位：以__号牙__为准__毫米
       {form['front_y_axis_position'][0]}，{cusp_name_dict[form['front_y_axis_position'][1]]}，{direction_name_dict[form['front_y_axis_position'][2]]}{form['front_y_axis_position'][3]:.1f}
    2. 垂直向目标位：后牙不动，前牙__毫米
       {direction_name_dict[form['z_axis_position']['front_teeth'][0]]}{form['z_axis_position']['front_teeth'][1]:.1f}

    上下颌咬合关系
    1. 咬合关系【左侧】：尖牙到__类关系，磨牙到__类关系
       一，一
    2. 咬合关系【右侧】：尖牙到__类关系，磨牙到__类关系
       一，一
    3. 咬合关系【覆盖】：__覆盖
       标准
    4. 咬合关系【覆颌】：__覆颌，__后牙
       标准，维持
    5. 咬合关系【后牙反颌|锁颌】
       纠正

    中线目标位
    1. [上颌] 中线目标位
       {direction_name_dict[form['middle_line_position']['U'][1]]}{form['middle_line_position']['U'][2]:.1f}毫米
    2. [下颌] 中线目标位
       对齐上颌中线

    碰撞解除|间隙闭合
    1. 上颌
       {collision2str(form['collision_removal']['U']['1'])}
    2. 下颌
       {collision2str(form['collision_removal']['L']['1'])}
    3. 拔牙牙号
       {'，'.join(map(str, form['remove_teeth_set']))}
    4. 预留间隙
       {'，'.join([f'{k}={v}' for k, v in form['init_gap'].items()]) if len(form['init_gap']) > 0 else '无'}
    """

    return form_str

print(form2str(json.loads(result['form'])))

以下输出为自动排牙结果

In [ ]:
sum(arranged_comp.values(), None).show()

## 自动分步方案


In [ ]:
# 定义调用JSON的基础部分 - 调用的服务名称以及您的用户名信息
json_call = {
  "spec_group": "mesh-processing", # 调用的工作流组， 随API文档发送
  "spec_name": "auto-step", # 调用的工作流名称， 随API文档发送, 
  "spec_version": "1.0-snapshot", # 调用的工作流版本，随API文档发送
  "user_group": user_group,
  "user_id": user_id
}
json_call["input_data"] = { # 输入数据，此处依照不同的任务字典内容会有不同，具体参照API文档，该处以分牙+牙轴举例
    "upper_teeth_dict": result['teeth_comp_u'],
    "upper_align_matrix": result['u_align_matrix'],
    "upper_axis_matrix_dict": result['u_axis'],
    "lower_teeth_dict": result['teeth_comp_l'],
    "lower_align_matrix": result['l_align_matrix'],
    "lower_axis_matrix_dict": result['l_axis'],
    "transformation_dict": result['transformation_dict']
}

headers = {
  "Content-Type": "application/json",
  "X-ZH-TOKEN": zh_token
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

# 打印任务号
print(run_id)

等待请求完成

In [ ]:
# 第二步： 轮询状态
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 600: # 最多等10分钟
    time.sleep(3) # 轮询间隔
    response = requests.request("GET", url, headers=headers)
    step_result = response.json()
    if step_result['completed'] or step_result['failed']:
        break
    
if not step_result['completed']:
    if step_result['failed']:
        raise ValueError("API运行错误，原因： " + str(step_result['reason_public']))
    raise TimeoutError("API超时")

print("API运行时间： {}s".format(time.time()-start_time))

In [ ]:
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
step_result = response.json()

分步结果展示

In [ ]:
def show_step(step_index):
    result_mesh = None
    for k, m in original_comp.items():
        if k in step_result['result']['step_dict'][step_index]:
            result_mesh += m.copy().apply_transform(step_result['result']['step_dict'][step_index][k])
    return result_mesh

In [ ]:
# 第50步展示
show_step(50).show()